# Simulatie Analyse

Er zal een statistische analyse worden op verschillende data die we hadden gekregen van meerdere batchruns met verschillende parameters. Aan de hand van deze data kunnen we onze onderzoeksvraag beantwoorden.

In [ ]:
# Libraries importeren 
from Classes.Person import Person
import pickle
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Dataframe importeren 
We gaan twee dataframes bekijken van onze simulatie met de volgende params:
* VoterModel
* 1000 voters
* 4 kandidaten 
* voter_type : Voter Type (Agent gedrag): [Strategisch, Honest]
* maxpolls: 6
* loyalty : 30
* strat_chance: 30 
* width:2
* height:2
* 1000 iterations/runs
* 10 steps 

Er is gekozen vor een loyalty en een strat_chance van 30%, want wij denken dat dit een realistische beeld  even van zo'n verkiezing.

In [ ]:
honest = pickle.load(open("generated_files/batch_run_honest_1","rb"))
strategic = pickle.load(open("generated_files/batch_run_strategic_1","rb"))

In [ ]:
honest

In [ ]:
def getFluctuations(pollList):
    """
    Berekent de gemiddelde verandering van de votes van een kandidaat per poll.
    params:
    pollList: lijst met de resultaten van elke peiling in een run.
    """
    fluctuation = {}
    length = len(pollList)
    for cand in pollList[0]:
        fluctuation.update({cand: 0})
                    
    for cand in fluctuation:
        differences = []
        for pollId in range(length-1):
            differences.append(pollList[pollId + 1].get(cand) - pollList[pollId].get(cand))
        fluctuation.update({cand:(sum(differences) / length)})

    return fluctuation

In [ ]:
def convertRuns(_data):
    """
    In deze functie gaat de data verwerkt worden waardoor het overzichtlijk is en beter leesbaare en te gebruiken.
    """
    runs = pd.DataFrame(columns=['run_id', 'cand_unique_id', 'cand_votes', 'cand_position', 'strat_perc'])
    for i, run in _data.iterrows():
        cand_fluctuations = getFluctuations(run.Polls) # bepaal de gemiddelde aantal votes per poll per kandidaat
        for cand in run.Polls[len(run.Polls)-1]:
            runs = runs.append({'run_id':i,'cand_unique_id':cand.unique_id,'cand_votes':run[1][0].get(cand),'cand_position':cand.position,'strat_perc': run[2],'fluctuation':cand_fluctuations.get(cand)},ignore_index=True)
    runs['cand_votes'] = runs['cand_votes'].convert_dtypes(int)
    return runs

In [ ]:
honest_runs = convertRuns(honest)

In [ ]:
strategic_runs = convertRuns(strategic)

In [ ]:
honest, strategic = None, None

In [ ]:
def addDistance(df:pd.DataFrame):
    """
    Afstand wordt berekent van de kandidaat ene de midden van de space.
    Dit zal gelijk zijn aan de coordinaten (1.0, 1.0)
    """
    df['distance'] = df['cand_position'].apply(lambda x: np.linalg.norm([1.0,1.0] - x))
    return df

In [ ]:
honest_runs = addDistance(honest_runs)
display(honest_runs.sort_values(by=['distance']))
#print de honest_Runs die gesorteerd zijn op de hoogste naar de laagste cand_votes
display(honest_runs.sort_values(by=['cand_votes'],ascending=False))

strategic_runs = addDistance(strategic_runs)
display(strategic_runs.sort_values(by=['distance']))
##print de honest_Runs die gesorteerd zijn op de hoogste naar de laagste cand_votes
display(strategic_runs.sort_values(by=['cand_votes'],ascending=False))

Aan de hand van pairplots gaan we de realities zien tussen de cand_votes en distance bij honest_runs. Bovendien gaan we de relatie zien van *'cand_votes','distance','strat_perc','fluctuation'* bij strategic_runs

In [ ]:
display(sns.pairplot(honest_runs[['cand_votes','distance']]))

In [ ]:
display(sns.pairplot(strategic_runs[['cand_votes','distance','strat_perc','fluctuation']]))

In [ ]:
# display(honest_runs[['cand_votes']].describe(include='all'))
# display(honest_runs[['cand_votes']].plot.box())
# display(strategic_runs[['cand_votes']].describe(include='all'))
# display(strategic_runs[['cand_votes']].plot.box())

# Effect van plurality voting op mensen dat niet op hun eerste voorkeur kiest.

Hier laten we een scatter plot zien met de gemiddelde fluctuatie in stemmen per pijlingsronden.
Wat hier uit te zien is is dat de kans dat je stemmen toe krijgt van strategische stemmers later in het stem proces steeds kleiner wordt deste verder je van het midden van het politeke centrum af staat.
<br><br>
De X-as van dit model laat de afstand van een kandidaad naar het centrum zien.<br>
De Y-as laat hier de hoeveelheid stemmen zien die een kandidaat krijg/verliest gemiddeld per pijling.

In [ ]:
plt.figure()
plt.scatter(data=strategic_runs, x='distance',y='fluctuation')
plt.hlines(y=0, xmin = 0.0, xmax = 1.4, color='r')

plt.title("Verandering van de aantal votes bij een kandidaat per poll")
plt.ylabel("toename/afname stemmers bij een kandidaat per poll")
plt.xlabel("Afstand van de kandidaat en de midden van de plane")
plt.show()

Ons model gebruikt meerdere parameters om te bepalen wanneer een stemmer een strategische stem wilt uitbrengen.
<br>
Hier is goed te zien dat er soms nog wat uitschieters zijn en er veel stemmers strategisch gaan stemmen, maar veel belangrijker dat de mediaan mooi rond de 10% ligt.
<br>
Zoals uit bronnen naar voren komt is het uiteidenlijke stemgedrag van mensen 9% strategisch.

In [ ]:
strategic_runs[['strat_perc']].describe()

De frequentie verdeling van fluctuatie.<br>

In [ ]:
count = strategic_runs.groupby([pd.cut(strategic_runs['distance'],12)])[['fluctuation']].count().reset_index()
ax =count[['fluctuation']].plot.bar(stacked=True)
ax.set_title("Frequentieverdeling van de fluctuatie van de aantal votes")

Hier is een Barplot te zien waar een gemiddelde waarde van de fluctuatie te zien is in verschillende bins.<br>
We hebben gekozen voor 12 bins.<br>

In [ ]:
sums = strategic_runs.groupby([pd.cut(strategic_runs['distance'],12)])[['fluctuation']].mean().reset_index()
ax = sums.plot.bar(x='distance',y='fluctuation',rot=90)
ax.set_title("Aantal votes een bepaalde kandidat kan krijgen/verliezen t.o.v de afstand tot de midden van de space ")
ax.set_xlabel("intervallen van de afstand tot de midden van een kandidaat")
ax.set_ylabel("Votes krijgen/verliezen per poll")

Zo is er duidelijk te zien dat een kandidaad die dichter bij het centrum zit over het algemeen meer stemmen krijgt mochten mensen besluiten strategisch te gaans stemmen.<br>
Het omkeer punt zit rond de 0.7 afstand.

# Effect van peilingen op het stemgedrag van stemmers met plurality voting.

Voor deze onderzoeksvraag gaan we kijken naar de effect van de aantal peilingen in een verkiezing op het stemgedrag van stemmers ten opzichte van de type kandidaat binnen de space met Plurality voting. Voor deze experiment hadden we aanname genomen dat een voting process ongeveer 2 weken zal duren. 
Om de effect te bepalen van de peilingen op het stemgedrag gaan we verschillende batch runs uitvoeren waar we gaan kijken naar de verkiezingen met de volgende aantal polls:
- 1 polls : Er wordt 1 peiling in de verkiezieng gepubliceerd
- 2 polls: Er wordt 2 peiling in de verkiezieng gepubliceerd ( on
- 6 polls: Er wordt 6 peiling in de verkiezing gepubliceerd( ongeveer 3 poll per week
- 14 polls: Er wordt dagelijks een peiling gepubliceerd.

In [ ]:
# Load dataframes
raw_poll_1 = pickle.load(open("generated_files/batch_run_strategic_2_1poll","rb"))
raw_poll_2 = pickle.load(open("generated_files/batch_run_strategic_2_2poll","rb"))
poll_6 = strategic_runs
# raw_poll_14 = pickle.load(open("generated_files/batch_run_strategic_2_14poll","rb"))

In [ ]:
# Convert dataframes
poll_1 = convertRuns(raw_poll_1)
poll_1 = addDistance(poll_1)


poll_2 = convertRuns(raw_poll_2)
poll_2 = addDistance(poll_2)

# poll_14 = convertRuns(raw_poll_14)
# poll_14 = addDistance(poll_14)

Om het effect van peilingen te bekijken op het stemgedrag van de stemmers, gaan we onze kolom distance verdelen in 5 groepen. Hier kunnen we 5 type kandidaat in onze space definieren. De gemiddelde aantal stemmers per bins/type kandidaat
wordt bepaald en gevisualiseerd in een bar plot. 

In [ ]:
poll_1_mean = poll_1.groupby([pd.cut(poll_1['distance'],12)])[['cand_votes']].mean()
poll_1_mean['stdev_cand_votes'] = poll_1.groupby([pd.cut(poll_1['distance'],12)])[['cand_votes']].std()
poll_1_mean = poll_1_mean.reset_index()
ax= poll_1_mean.plot.bar(x= 'distance',y='cand_votes',rot=90)
ax.set_title("Gemiddelde votes per type kandidaat bij verkiezingen met 1 peilingen")

In [ ]:
poll_2_mean = poll_2.groupby([pd.cut(poll_2['distance'],12)])[['cand_votes']].mean()
poll_2_mean['stdev_cand_votes'] = poll_2.groupby([pd.cut(poll_2['distance'],12)])[['cand_votes']].std()
poll_2_mean = poll_2_mean.reset_index()
ax= poll_2_mean.plot.bar(x= 'distance',y='cand_votes',rot=90)
ax.set_title("Gemiddelde votes per type kandidaat bij verkiezingen met 2 peilingen")

In [ ]:
poll_6_mean = poll_6.groupby([pd.cut(poll_6['distance'],12)])[['cand_votes']].mean()
poll_6_mean['stdev_cand_votes'] = poll_6.groupby([pd.cut(poll_6['distance'],12)])[['cand_votes']].std()
poll_6_mean = poll_6_mean.reset_index()
ax= poll_6_mean.plot.bar(x= 'distance',y='cand_votes',rot=90)
ax.set_title("Gemiddelde votes per type kandidaat bij verkiezingen met 6 peilingen")

In [ ]:
poll_14_mean = poll_14.groupby([pd.cut(poll_14['distance'],5)])[['cand_votes']].mean()
poll_14_mean['stdev_cand_votes'] = poll_14.groupby([pd.cut(poll_14['distance'],5)])[['cand_votes']].std()
poll_14_mean = poll_14_mean.reset_index()
ax= poll_14_mean.plot.bar(x= 'distance',y='cand_votes',rot=90)
ax.set_title("Gemiddelde votes per type kandidaat bij verkiezingen met 14 peilingen")

In [ ]:
poll_1_mean['colId'] = 1
poll_2_mean['colId'] = 2
poll_6_mean['colId'] = 6
poll_14_mean['colId'] = 14
allPolls = poll_1_mean.append(poll_2_mean)
allPolls = allPolls.append(poll_6_mean)
allPolls = allPolls.append(poll_14_mean)

sns.lineplot(data=allPolls,x=allPolls.index, y="cand_votes", hue="colId")
ax2 = plt.twinx()
sns.lineplot(data=allPolls, x=allPolls.index, y=allPolls.stdev_cand_votes, ax=ax2)

Aan de hand van de barplot en de bovenstaande data kunnen we concluderen dat kandidaten met een *neutrale politieke visie* hebben meer kans om stemmers te krijgen.Bovendien is te zien dat hoe meer peilingen er wordt gepubliceerd, kandidaten krijegen en verliezen meer stemmers gedurende zo'n verkiezing.

Hoe meer peilingen, hoe meer kans een stemmer heeft om een strategie te gebruiken en zijn keuze veranderen.
De aantal peilingen heeft dus een bepaalde invloed op de stemmers, waar ze voor elke poll meer meer open staan om te stemmen voor een kandidaat buiten hun politieke standpunten.